## RaspberryPi als Wetterstation

Erstellt am 24.10.2018 für Raspberry Pi 3 Model B.

# Ziel
**Wir werden eine Wetterstation bauen, basierend auf dem Bosch-Sensor BME680, welcher Lufttemperatur, Luftdruck und relative Luftfeuchte misst. Die Darstellung der Werte erfolgt übers Handy durch ansurfen einer Web-Page auf dem RaspberryPi.**

Hierfür sind folgende Schritte notwendig:

1. Installation Linux Betriegssystem und den notwendigen Programmen (Python, Spyder,...)
2. Hardware-Konfiguration um die Schnittstelle zum BME-Sensor (I2C) und für Modubus (Seriell-UART) freizuschalten
3. Konfiguration Remote-Zugriff für Dateien (SFTB) und Fernwartung (RealVNC)
4. Web-Zugriff über Django-Framework

## 1. Installation
Mit SDformatter (geht auch für Windows10) die SD-Karte formatieren.

**Installation dauert über 1 Stunde!** Von [www.raspberrypi.org](http://www.raspberrypi.org) aktuelles NOOBS „Network install only“ (nicht offline) downloaden. Voraussetzung ist Netzwerk-Zugriff. Anschliessend auf SD-Karte entpacken.

Dann SD-Karte in RaspberryPi einsetzen und starten. Zuerst Sprache Deutsch und Tastatur CH eingeben, dann WLAN verbinden. **Raspbian with PIXEL (Desktop Version)** ankreuzen, und „Installieren“ klicken. 

Nach Installation erfolgt ein Neustart und Konfiguration, wie z.B. Ort, Zeitzone, Passwort.

    
Wir arbeiten mit Python 3.x. Um Bibliotheken hierfür zu installieren muss darauf geachtet werden das die "3"-Version verwendet wird. Z.B. **pip3 anstatt pip** oder spyder3 anstatt spyder. Ohne "3" wird die Version für Python 2.x verwendet. **Der Raspberry benötigt für das System beide Versionen!** Standardmässig wird Python2.x verwendet. Erkennbar durch "python --version", oder man startet auf der Console "python" und führt die Division 3/2 durch. Python2.x liefert 1. Wird auf der Konsole "python3" gestartet ergibt 3/2 = 1.5.

Anschliessend werden einige Programme installiert (dauert > 30 Minuten). Hierfür verwenden wir den  Linux-Package-Installer "apt-get3" für die Entwicklungsumgebung Spyder:

    sudo apt-get install spyder3 -y

Für die Python-Bibliotheken verwenden wir den Python-Package-Installer "pip3":

    sudo pip3 install numpy matplotlib django bme680
    
Dies sind die Python3-Bibliotheken und Django für die Web-Visualiserung, sowie BME680 für den Bosch-Sensor.

Für einen Eindruck, was mit dem Bosch-Sensor möglich ist, siehe https://learn.pimoroni.com/tutorial/sandyj/getting-started-with-bme680-breakout.

## 2. Hardware-Konfiguration
Die serielle Schnittstelle UART für Modbus ist auf den GPIO 14, 15 (Pin 8, Pin 10 des Headers).
Die I2C-Kommunikation für den Bosch-Sensor ist auf den GPIO 2, 3 (Pin 3, Pin 5 des Headers). 

![Pin-Belegung des Headers vom RaspberryPi](rpiGPIO.JPG)

Weiters benötigen wir Remote-Zugriff. Dies erfolgt über **Einstellungen>Raspberry-Pi-Konfiguration>Schnittstellen** mit folgenden Anpassungen:
 - aktivieren SSH (für Datei-Zugriff mit SFTP)
 - aktivieren VNC (für Remote-Desktop-Zugriff)
 - aktivieren I2C (für GPIO-Schnittstelle zum BME680 Sensor)
 - aktivieren Serial Port (für Modbus-Kommunikation mit RS485)
 - DEaktivieren Serial Console
 
Lege die Bildschirm-Auflösung fest über **Einstellungen>Raspberry-Pi-Konfiguration>System** hier der Button: "Auflösung festlegen".
 

Raspberry Neustarten (auf Console: sudo reboot).


### Serielle Schnittstelle für GPIO mit Subplatine
MODBUS-RTU basiert auf RS485 und kann mit dem Raspberry Pi über eine USB-RS485 Schnittstelle angeprochen werden. Wir verwenden die Subplatine "WeatherShild" (hergestellt an der NTB), welche die Anbindung über die GPIOs des RPI macht. Diese ist aber standardmässig für Bluetooth (bt) vorgesehen, deshalb deaktivieren wir zuerst bt, sodass die uart für die GPIO 14, 15 frei ist.

Hierfür ist eine Anpassungen mit einem Text-Editor, in einer Konfigurations-Datei notwendig, zuerst aber "wie öffne ich ein Editor?" Terminal öffnen uns folgende Eingabe `sudo geany`

 - In der Datei ** /boot/config.txt ** am Ende der Datei folgenden Befehl ergänzen:
`dtoverlay=pi3-disable-bt`

Anschliessend speichern und Neustarten:

    sudo reboot

Nun müsste das Bluetooth-Symbol rechts oben deaktiv sein. Die Schnittstelle ist über Python erreichbar mit *** "dev/ttyAMA0" """.

## 3. Remotezugriff
Remotezugriff erfolgt über RealVNC-Viewer. Dieser muss auf dem PC installiert werden (download vom Internet). Für den Zugriff ist IP-Adresse, User-Name und Passwort notwendig.

IP-Adresse des RaspberryPi ermittelt man mit Konsole-Eingabe:I

    ifconfig

Prüfe zuerst auf dem PC-Terminal ob eine Verpindung zum RaspberryPi vorhanden ist mit

    ping [ip-Adresse]

Username: pi, Password: ??oben gewähltes passwort??


### File Transfer SFTP 
Um bestehende py-Skripts vom PC auf den RaspberryPi zu übertragen benötigen wir ein SFTP-Programm. Programme hierfür sind z.B. WinSCP, FileZilla, usw...

Für einen Zuriff auf den RaspberryPi sind Zugriffsdaten notwendig: **Port: 22 (wichtig, weil SFTP)**, Username: pi, Password: ??oben gewähltes passwort?? und der Name des Host/Server, dies ist die IP-Adresse.

## 4. Django
Als Ausgangspunkt verwenden wir das Kurs-Template von https://github.com/donmiguel/energieDigital. Dieses laden wir herunter und entpacken und probieren es zuerst auf dem PC, hier jedoch ohne Sensoren.

Zuerst wird das Skript-File "daq.py" gestartet, abgelegt unter /energieDigital-master/chartGenerator/daq.
Dieses liesst zyklisch die Sensoren aus und speichert die Werte als csv-Datei im selben Ordner ab. 

Das Auslesen der Sensoren wird ebenfalls in dienser Datei konfiguriert, mit den Variablen:

    hukseConnect = True
    bmeConnect   = True
    
Das starten von Django erfolgt mit:

    python manage.py runserver
    
Nun können wir im Browser die Applikation mit http://127.0.0.1:8000/cg/example starten.

Erfolgt der Start mit der IP-Adresse

    python manage.py runserver [IP-Adresse]:8000 

kann die Webpage auch von einem anderen Gerät im Netzwerk aufgerufen werden, sofern das Netzwerk (Firewall) den Port 8000 erlaubt.

Wenn es auf dem PC funktioniert, kann das Projekt auf den RaspberryPi in das "pi"-Verzeichnis kopiert werden. 

## Nicht behandelt wird...
### Virtaul Enviroment
Für eine umfangreichere Entwicklung ist der Einsatz einer Virtuellen Umgebung ratsam,jedoch werden wir dies nicht behandeln.

Allgemein bei SW, wie auch bei Python und Bibliotheken, soll immer mit den aktuellsten Versionen gearbeitet werden, somit muss man sich nicht mit bereits gelösten Problemen auseinander setzen. Es macht aber auch Sinn mit älteren Modulen zu arbeiten. Wenn eine eigene Funktion oder Anwendung erstellt wird, so möchte man sicherstellen, dass diese auch in einigen Jahren noch funktioniert ohne weiteren Entwicklungaufwand, d.h. das System und die Pakete, mit welchen entwickelt wurde sollen "konserviert" werden. Dies ist mit Virtual Enviroment (VE) möglich. Das Gesamtsystem kann aktuell gehalten werden und in einem "Verzeichnis" wird ein Entwicklungsstand "konserviert".

### Production Server
Für eine produktiven Einsatz der Web-Page ist ein leistungsfähiger Server wie Apache notwendig. Wir werden diesen jedoch nicht installieren. Wir verwenden für Djanog den Django-internen Entwicklungsserver. 
